import torch
from torch import nn
import matplotlib.pyplot as plt

torch.manual_seed(0)
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())

## 1) SwiGLU 简介

SwiGLU 是 **Swish 激活函数 + GLU 门控线性单元** 的组合，是 LLM 中最常用的 Feed-Forward 架构之一。

**传统 FFN**: `FFN(x) = W2 * Act(W1 * x)`

**SwiGLU**: `SwiGLU(x) = (W1 * x) ⊙ (Act(W2 * x))`  ->  `W3 * output`

其中 ⊙ 表示逐元素乘法，Act 通常是 **SiLU/Swish**：`silu(x) = x * sigmoid(x)`

In [ ]:
def silu(x):
    """SiLU / Swish 激活函数: x * sigmoid(x)"""
    return x * torch.sigmoid(x)

def gelu(x):
    """GELU 激活函数 (近似实现)"""
    return 0.5 * x * (1.0 + torch.tanh(torch.sqrt(2.0 / torch.tensor(3.1415926535)) * (x + 0.044715 * x ** 3)))

# 对比可视化
x = torch.linspace(-4, 4, 200)

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(x.numpy(), silu(x).numpy(), label="SiLU (Swish)", linewidth=2)
plt.plot(x.numpy(), torch.relu(x).numpy(), label="ReLU", linewidth=2, linestyle="--")
plt.axhline(0, color="gray", alpha=0.3)
plt.axvline(0, color="gray", alpha=0.3)
plt.title("激活函数对比")
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(x.numpy(), silu(x).derivative().numpy(), label="SiLU 导数", linewidth=2)
plt.plot(x.numpy(), (x > 0).float().numpy(), label="ReLU 导数", linewidth=2, linestyle="--")
plt.title("导数对比")
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("SiLU 特点: 平滑、非单调、有负值区域（相比ReLU有更好的梯度流）")

## 2) GLU 门控机制

GLU (Gated Linear Unit) 的核心思想：
1. 用一个线性变换 `gate_proj` 生成门控信号
2. 门控信号经过激活函数后，与另一个线性变换 `up_proj` 的输出做**逐元素乘法**
3. 最后用 `down_proj` 投影回原维度

数学表达式：
$$
\text{SwiGLU}(x) = W_3 \left( (W_1 x) \odot \text{SiLU}(W_2 x) \right)
$$

In [ ]:
class FeedForward(nn.Module):
    """SwiGLU Feed Forward Network"""
    def __init__(self, hidden_size: int, intermediate_size: int = None, dropout: float = 0.1):
        super().__init__()
        if intermediate_size is None:
            # 默认: intermediate_size ≈ hidden_size * 8/3 (参考 LLaMA)
            intermediate_size = int(hidden_size * 8 / 3)
            intermediate_size = 64 * ((intermediate_size + 64 - 1) // 64)  # 对齐到 64
        
        self.hidden_size = hidden_size
        self.intermediate_size = intermediate_size
        
        # 三个投影矩阵
        self.gate_proj = nn.Linear(hidden_size, intermediate_size, bias=False)  # W1 (门控)
        self.up_proj = nn.Linear(hidden_size, intermediate_size, bias=False)    # W2 (上投影)
        self.down_proj = nn.Linear(intermediate_size, hidden_size, bias=False)  # W3 (下投影)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # SwiGLU 前向传播
        # gate: [B, T, hidden_size] -> [B, T, intermediate_size]
        # up:   [B, T, hidden_size] -> [B, T, intermediate_size]
        # gate_act: 经过 SiLU 激活
        # output: gate_act * up -> down_proj -> [B, T, hidden_size]
        return self.dropout(self.down_proj(silu(self.gate_proj(x)) * self.up_proj(x)))

# 参数统计
def count_parameters(ffn: FeedForward) -> dict:
    """统计参数数量"""
    params = {
        "gate_proj": sum(p.numel() for p in ffn.gate_proj.parameters()),
        "up_proj": sum(p.numel() for p in ffn.up_proj.parameters()),
        "down_proj": sum(p.numel() for p in ffn.down_proj.parameters()),
    }
    params["total"] = sum(params.values())
    return params

# 测试
hidden_size = 4096
ffn = FeedForward(hidden_size=hidden_size)
params = count_parameters(ffn)

print(f"hidden_size: {hidden_size}")
print(f"intermediate_size: {ffn.intermediate_size}")
print(f"参数统计:")
for k, v in params.items():
    print(f"  {k}: {v:,} ({v/1e6:.2f}M)")
print(f"\n对比传统 FFN (2个 linear): {2 * hidden_size * ffn.intermediate_size + ffn.intermediate_size * hidden_size:,}")
print(f"SwiGLU (3个 linear): {params['total']:,}")

## 3) 为什么 SwiGLU 效果好？

### 3.1 门控机制的优势
- SiLU 作为门控：`gate = SiLU(W1 x)`，输出范围约 (-0.28, ∞)
- 负值可以**抑制**某些特征通道，正值可以**增强**某些特征
- 相比 ReLU 的硬截断，SiLU 更平滑，梯度更稳定

### 3.2 逐元素乘法的信息交互
- `gate * up` 让两个不同的线性变换结果相互作用
- 相当于一种**自适应特征选择**机制

### 3.3 参数量与计算量的权衡
- SwiGLU 比传统 FFN 多一个投影，但中间层更大
- 整体参数量约为 3 * hidden_size * intermediate_size
- LLaMA 等模型选择 intermediate_size ≈ hidden_size * 8/3，兼顾容量与效率

In [ ]:
# 可视化门控效果
batch_size, seq_len, hidden_size = 2, 10, 4096
intermediate_size = ffn.intermediate_size

x = torch.randn(batch_size, seq_len, hidden_size)

# 前向传播，获取中间结果
gate = ffn.gate_proj(x)           # [B, T, intermediate_size]
gate_activated = silu(gate)       # 经过 SiLU 激活
up = ffn.up_proj(x)               # [B, T, intermediate_size]
gated = gate_activated * up       # 逐元素乘法
output = ffn.down_proj(gated)     # [B, T, hidden_size]

print(f"输入形状: {x.shape}")
print(f"gate_proj 输出形状: {gate.shape}")
print(f"up_proj 输出形状: {up.shape}")
print(f"门控后形状: {gated.shape}")
print(f"最终输出形状: {output.shape}")

# 分析门控信号分布
print("\n门控信号统计 (SiLU(gate)):")
print(f"  均值: {gate_activated.mean().item():.4f}")
print(f"  标准差: {gate_activated.std().item():.4f}")
print(f"  最小值: {gate_activated.min().item():.4f}")
print(f"  最大值: {gate_activated.max().item():.4f}")
print(f"  负值比例: {(gate_activated < 0).float().mean().item():.2%}")

# 可视化
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(gate_activated.flatten().cpu().numpy(), bins=50, alpha=0.7, edgecolor='black')
plt.axvline(0, color='red', linestyle='--', label='zero')
plt.title("门控信号 SiLU(gate) 分布")
plt.xlabel("值")
plt.ylabel("频次")
plt.legend()

plt.subplot(1, 2, 2)
plt.hist(up.flatten().cpu().numpy(), bins=50, alpha=0.7, color='orange', edgecolor='black')
plt.title("up_proj 输出分布")
plt.xlabel("值")
plt.ylabel("频次")

plt.tight_layout()
plt.show()

## 4) 梯度流分析

In [ ]:
# 对比传统 FFN 与 SwiGLU 的梯度流
class TraditionalFFN(nn.Module):
    """传统 FFN: ReLU(W2(W1 x))"""
    def __init__(self, hidden_size, intermediate_size):
        super().__init__()
        self.w1 = nn.Linear(hidden_size, intermediate_size, bias=False)
        self.w2 = nn.Linear(intermediate_size, hidden_size, bias=False)
        self.act = nn.ReLU()
        
    def forward(self, x):
        return self.w2(self.act(self.w1(x)))

class SwiGLUFFN(nn.Module):
    """SwiGLU FFN"""
    def __init__(self, hidden_size, intermediate_size):
        super().__init__()
        self.gate_proj = nn.Linear(hidden_size, intermediate_size, bias=False)
        self.up_proj = nn.Linear(hidden_size, intermediate_size, bias=False)
        self.down_proj = nn.Linear(intermediate_size, hidden_size, bias=False)
        
    def forward(self, x):
        return self.down_proj(silu(self.gate_proj(x)) * self.up_proj(x))

# 创建模型
hidden_size = 512
intermediate_size = 2048

ffn_trad = TraditionalFFN(hidden_size, intermediate_size)
ffn_swiglu = SwiGLUFFN(hidden_size, intermediate_size)

# 输入
x = torch.randn(4, 32, hidden_size, requires_grad=True)

# 传统 FFN 梯度
x_trad = x.clone()
y_trad = ffn_trad(x_trad)
loss_trad = y_trad.sum()
loss_trad.backward()
grad_trad = x_trad.grad.abs().mean(dim=-1)  # [B, T]

# SwiGLU 梯度
x_swiglu = x.clone()
y_swiglu = ffn_swiglu(x_swiglu)
loss_swiglu = y_swiglu.sum()
loss_swiglu.backward()
grad_swiglu = x_swiglu.grad.abs().mean(dim=-1)  # [B, T]

# 可视化
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(grad_trad[0].detach().cpu().numpy(), label="Traditional FFN (ReLU)", alpha=0.7)
plt.plot(grad_swiglu[0].detach().cpu().numpy(), label="SwiGLU", alpha=0.7)
plt.title("梯度强度对比 (第一个 batch)")
plt.xlabel("Token 位置")
plt.ylabel("平均梯度强度")
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(grad_trad.flatten().detach().cpu().numpy(), 
            grad_swiglu.flatten().detach().cpu().numpy(), alpha=0.1, s=1)
plt.plot([0, grad_trad.max()], [0, grad_trad.max()], 'r--', label="y=x")
plt.xlabel("Traditional FFN 梯度")
plt.ylabel("SwiGLU 梯度")
plt.title("梯度分布对比")
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"传统 FFN 平均梯度: {grad_trad.mean().item():.6f}")
print(f"SwiGLU 平均梯度: {grad_swiglu.mean().item():.6f}")
print("\n结论: SwiGLU 的门控机制使得梯度流更加平滑，减少了死神经元的问题")

## 5) 代码实现对比

In [ ]:
# PyTorch 实现
class PyTorchSwiGLU(nn.Module):
    """使用 nn.Module 实现 SwiGLU"""
    def __init__(self, hidden_size: int, intermediate_size: int):
        super().__init__()
        self.gate_proj = nn.Linear(hidden_size, intermediate_size, bias=False)
        self.up_proj = nn.Linear(hidden_size, intermediate_size, bias=False)
        self.down_proj = nn.Linear(intermediate_size, hidden_size, bias=False)
        self.silu = nn.SiLU()
        
    def forward(self, x):
        return self.down_proj(self.silu(self.gate_proj(x)) * self.up_proj(x))

# 使用 torch.nn.functional 实现
import torch.nn.functional as F

class FSwiGLU(nn.Module):
    """使用 F.silu 实现 SwiGLU"""
    def __init__(self, hidden_size: int, intermediate_size: int):
        super().__init__()
        self.gate_proj = nn.Linear(hidden_size, intermediate_size, bias=False)
        self.up_proj = nn.Linear(hidden_size, intermediate_size, bias=False)
        self.down_proj = nn.Linear(intermediate_size, hidden_size, bias=False)
        
    def forward(self, x):
        return F.silu(self.gate_proj(x)) * self.up_proj(x) @ self.down_proj.weight.t()

# 验证两种实现输出一致
hidden_size = 256
intermediate_size = 512

model1 = PyTorchSwiGLU(hidden_size, intermediate_size)
model2 = FSwiGLU(hidden_size, intermediate_size)

# 复制权重
model2.gate_proj.weight.data = model1.gate_proj.weight.data.clone()
model2.up_proj.weight.data = model1.up_proj.weight.data.clone()
model2.down_proj.weight.data = model1.down_proj.weight.data.clone()

# 测试
x = torch.randn(2, 10, hidden_size)
y1 = model1(x)
y2 = model2(x)

print(f"PyTorch 实现输出形状: {y1.shape}")
print(f"F.silu 实现输出形状: {y2.shape}")
print(f"输出差异: {(y1 - y2).abs().max().item():.2e}")

# 参数统计
total_params = sum(p.numel() for p in model1.parameters())
print(f"总参数量: {total_params:,}")

## 6) 总结

### SwiGLU 的核心特点
1. **门控机制**: SiLU 作为软门控，比 ReLU 更平滑
2. **逐元素交互**: gate * up 实现特征自适应选择
3. **参数效率**: 3个投影，但中间层更大（LLaMA: 8/3倍）
4. **梯度流**: 负值区域允许梯度流动，减少死神经元

### 实际应用
- LLaMA, PaLM, Mistral 等主流 LLM 都采用 SwiGLU
- 中间层大小通常是 hidden_size 的 8/3 倍，并向上取整到 64 的倍数